# Gauss-Hermite Approximation

### Load packages

In [1]:
from numpy.polynomial.hermite import hermgauss
from matplotlib import pyplot as plt
from numpy.linalg import inv
import pandas as pd
import numpy as np
import scipy
from scipy.stats import norm

### Randomized simulation data for two sites
- Set random seed
- Generate 10 true betas ranged in (-10, 10)
- Generate 2 sigmas for noise variance in different sites
- Generate data X1 and X2 with size (1000, 10)
- Generate result y1 and y2 with bernoulli distibution 

In [2]:
np.random.seed(30)
true_beta = (np.random.rand(10,1) - np.random.rand(10,1)) * 10
true_sigma = np.random.rand(2)
X1 = (np.random.rand(1000, 10) - np.random.rand(1000, 10)) * 10
p1 = 1 / (1 + np.exp(-(X1 @ true_beta + np.random.normal(0, true_sigma[0], 1000).reshape(1000, 1))))
y1 = np.random.binomial(1,p1)
X2 = (np.random.rand(1000, 10) - np.random.rand(1000, 10)) * 10
p2 = 1 / (1 + np.exp(-(X2 @ true_beta + np.random.normal(0, true_sigma[1], 1000).reshape(1000, 1))))
y2 = np.random.binomial(1,p2)

### Definitions

Notation for $\pi_{ij}$:

$$\pi_{ij} = \dfrac{\exp{(X_{ij}^\top\beta_0}+\mu_{i0})}{1 + \exp{(X_{ij}^\top\beta_0}+\mu_{i0})}$$

In [3]:
def Pi(x, beta_0, mu):
    return np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))))

Notation for $g(\mu_{i0})$:

$$g(\mu_{i0};\beta_0)=\sum_{j=1}^{n_i}\left[y_{ij}\log\pi_{ij}+(1-y_{ij})\log(1-\pi_{ij})\right]+\log\phi(\mu_{i0};\theta_0)$$

In [4]:
def g(x, y, mu, beta_0, tau=1):
    g = sum(y * np.log(Pi(x, beta_0, mu)) + (1 - y) * np.log(1 - Pi(x, beta_0, mu))) \
    + np.log((np.sqrt(2 * np.pi) * tau)**(-1) * np.exp(-mu**2/(2 * tau**2)))
    return g

In [6]:
# Plugged in log-sum-exp trick
def logsumexp(x):
    c = x.max()
    LSE = c + np.log(np.sum(np.exp(x - c)))
    return np.exp(x - LSE)

def g(x, y, mu, beta_0, tau=1):
    g = sum(y * logsumexp(Pi(x, beta_0, mu)) + (1 - y) * logsumexp(1 - Pi(x, beta_0, mu))) \
    + np.log((np.sqrt(2 * np.pi) * tau)**(-1) * np.exp(-mu**2/(2 * tau**2)))
    return g

Notation for $g_\beta(\mu_{i0})$:

$$\dfrac{\partial g}{\partial \beta_0}=\sum_{j=1}^{n_i}(X_{ij}y_{ij}-X_{ij}\pi_{ij})$$

In [7]:
def g_b(x, y, mu, beta_0, tau = 1):
    return np.sum(x * y - x * Pi(x, beta_0, mu), axis = 0)

Notation for $g_\mu(\mu_{i0})$:

$$\dfrac{\partial g}{\partial \mu_{i0}}=\sum_{j=1}^{n_i}(y_{ij}-\pi_{ij})-\dfrac{\mu_{i0}}{\tau_0^2}$$

In [8]:
def g_u(x, y, mu, beta_0, tau = 1):
    return sum(y - Pi(x, beta_0, mu)) - mu/tau**2

Notation for $g_{\mu\mu}(\mu_{i0})$:

$$\dfrac{\partial^2 g}{\partial \mu_{i0}^2}=-\sum_{j=1}^{n_i}\dfrac{\partial\pi_{ij}}{\partial\mu_{i0}}-\dfrac{1}{\tau_0^2}$$

In [9]:
def g_uu(x, y, mu, beta_0, tau = 1):
    return sum(- np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))**2))) - 1/tau**2

Notation for $g_{\mu\beta}(\mu_{i0})$:

$$\dfrac{\partial^2 g}{\partial \mu_{i0}\partial \beta_0}=-\sum_{j=1}^{n_i}\dfrac{\partial\pi_{ij}}{\partial\beta_0}$$

In [10]:
def g_ub(x, y, mu, beta_0, tau = 1):
    return np.sum(- x * np.asarray((np.exp(x @ beta_0 + mu) / (1 + np.exp(x @ beta_0 + mu))**2)), axis = 0)

Notation for $g_{\beta\beta}(\mu_{i0})$:

$$\dfrac{\partial^2 g}{\partial \beta_0^2}=-\sum_{j=1}^{n_i}X_{ij}\dfrac{\partial\pi_{ij}}{\partial\beta_0}$$

In [11]:
def g_bb(x, y, mu, beta_0, tau = 1):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1],1) @ x[i].reshape(1,x.shape[1])\
        * (np.exp(x[i] @ beta_0 + mu) / (1 + np.exp(x[i] @ beta_0 + mu))**2))
    return result

Notation for $g_{\mu\beta}(\hat\mu_{i0};\beta_0)$:

$$\dfrac{\partial g_u(\hat\mu(\beta_0);\beta_0)}{\partial \beta_0}=\hat\mu_\beta(\beta_0)g_{\mu\mu}(\hat\mu_{i0})+g_{\mu\beta}(\hat\mu_{i0})=0$$

Notation for $g_{\mu\mu\mu}(\hat\mu_{i0};\beta_0)$:

$$g_{\mu\mu\mu}(\hat\mu_{i0};\beta_0) = -\sum_{j=1}^{n_i}\dfrac{\partial^2\pi_{ij}}{\partial\mu_{i0}^2}$$

In [12]:
def g_uuu(x, y, mu, beta_0, tau = 1):
    return sum(- np.asarray((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3)))

Notation for $g_{\mu\mu\beta}(\hat\mu_{i0};\beta_0)$:

$$g_{\mu\mu\beta}(\hat\mu_{i0};\beta_0) = -\sum_{j=1}^{n_i}\dfrac{\partial^2\pi_{ij}}{\partial\mu_{i0}\partial\beta_0}$$

In [13]:
def g_uub(x, y, mu, beta_0, tau = 1):
    return np.sum(- x * np.asarray((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3)), axis = 0)

Notation for $g_{\mu\beta\beta}(\hat\mu_{i0};\beta_0)$:

$$\dfrac{\partial^3 g}{\partial \mu_{i0}\partial \beta_0^2}=-\sum_{j=1}^{n_i}\dfrac{\partial^2\pi_{ij}}{\partial\beta_0^2}$$

In [14]:
def g_ubb(x, y, mu, beta_0, tau = 1):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1],1) @ x[i].reshape(1,x.shape[1])\
                         * (np.exp(x[i] @ beta_0 + mu) * (np.exp(x[i] @ beta_0 + mu) - 1)\
                                       / (1 + np.exp(x[i] @ beta_0 + mu))**3))
    return result

Notation for $g_{\mu\mu\mu\mu}(\hat\mu_{i0};\beta_0)$:

$$g_{\mu\mu\mu\mu}(\hat\mu_{i0};\beta_0) = -\sum_{j=1}^{n_i}\dfrac{\partial^3\pi_{ij}}{\partial\mu_{i0}^3}$$

In [15]:
def g_uuuu(x, y, mu, beta_0, tau = 1):
    result = sum(- (np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3)\
                 + (3 * np.exp(2 * (x @ beta_0 + mu)) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**4)\
                 - (np.exp(2 * (x @ beta_0 + mu))  / (1 + np.exp(x @ beta_0 + mu))**3)\
                )
    return result

Notation for $g_{\mu\mu\mu\beta}(\hat\mu_{i0};\beta_0)$:

$$g_{\mu\mu\mu\beta}(\hat\mu_{i0};\beta_0) = -\sum_{j=1}^{n_i}\dfrac{\partial^3\pi_{ij}}{\partial\mu_{i0}^2\partial\beta_0}$$

In [16]:
def g_uuub(x, y, mu, beta_0, tau = 1):
    result = np.sum(- x * np.asarray((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3)\
                 + (3 * np.exp(2 * (x @ beta_0 + mu)) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**4)\
                 - (np.exp(2 * (x @ beta_0 + mu))  / (1 + np.exp(x @ beta_0 + mu))**3)), axis = 0)
    return result

Notation for $g_{\mu\mu\beta\beta}(\hat\mu_{i0};\beta_0)$:

$$g_{\mu\mu\beta\beta}(\hat\mu_{i0};\beta_0) = -\sum_{j=1}^{n_i}\dfrac{\partial^3\pi_{ij}}{\partial\mu_{i0}\partial\beta^2_0}$$

In [17]:
def g_uubb(x, y, mu, beta_0, tau = 1):
    result = 0
    for i in range(len(y)):
        result += -np.asarray(x[i].reshape(x.shape[1],1) @ x[i].reshape(1,x.shape[1])\
                             * ((np.exp(x @ beta_0 + mu) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**3)\
                 + (3 * np.exp(2 * (x @ beta_0 + mu)) * (np.exp(x @ beta_0 + mu) - 1)\
                             / (1 + np.exp(x @ beta_0 + mu))**4)\
                 - (np.exp(2 * (x @ beta_0 + mu))  / (1 + np.exp(x @ beta_0 + mu))**3)))
        
    return result

Notation for $\hat\omega$:

$$\hat\omega=\sqrt{-\dfrac{1}{g_{\mu\mu}(\hat\mu_{i0})}}$$

In [18]:
def omega(x, y, mu, beta_0, tau = 1):
    return np.sqrt(-1/g_uu(x, y, mu, beta_0))

Notation for $\hat\omega_\beta$:

$$\dfrac{\partial\hat\omega}{\partial\beta}=\dfrac{1}{2}\hat\omega^3(g_{\mu\mu\mu}\hat\mu_\beta + g_{\mu\mu\beta})$$

In [19]:
def omega_b(x, y, mu, beta_0, tau = 1):
    return 0.5 * omega(x, y, mu, beta_0, tau)**3 * (g_uuu(x, y, mu, beta_0, tau) * mu_b(x, y, mu, beta_0, tau)\
                                                    + g_uub(x, y, mu, beta_0, tau))

Notation for $\hat\mu_\beta$

$$\hat\mu_\beta(\beta_0)=-\dfrac{g_{\mu\beta}(\hat\mu_{i0};\beta_0)}{g_{\mu\mu}(\hat\mu_{i0};\beta_0)}=\hat\omega^2(\beta_0)g_{\mu\beta}(\hat\mu_{i0}(\beta_0);\beta_0)$$

In [20]:
def mu_b(x, y, mu, beta_0, tau = 1):
    return omega(x, y, mu, beta_0, tau)**2 * g_ub(x, y, mu, beta_0, tau)

Notation for $\hat\mu_{\beta\beta'}$:

$$\hat\mu_{\beta\beta'}(\beta_0)=\hat\omega^2(\hat\mu_\beta\hat\mu_{\beta'} g_{\mu\mu\mu}+\hat\mu_{\beta}g_{\mu\mu\beta'}+\hat\mu_{\beta'}g_{\mu\mu\beta}+g_{\mu\beta\beta'})$$

In [21]:
def mu_bb(x, y, mu, beta_0, tau = 1):
    result = omega(x, y, mu, beta_0, tau)**2 * (mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1)\
                                              @ mu_b(x, y, mu, beta_0, tau).reshape(1,x.shape[1])\
                                              * g_uuu(x, y, mu, beta_0, tau)\
                                              + 2 * mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1)\
                                              @ g_uub(x, y, mu, beta_0, tau).reshape(1,x.shape[1])\
                                              + g_ubb(x, y, mu, beta_0, tau))
    return result

Notation for $\hat\omega_{\beta\beta'}$:

$$\dfrac{\partial^2}{\partial\beta\partial\beta'}\hat\omega(\beta_0)=\frac{3}{4}\hat\omega^5(\hat\mu_{\beta'}g_{\mu\mu\mu}+g_{\mu\mu\beta'})(\hat\mu_{\beta}g_{\mu\mu\mu}+g_{\mu\mu\beta})+\dfrac{1}{2}\hat\omega^3(\hat\mu_{\beta\beta'}g_{\mu\mu\mu}+\hat\mu_\beta\hat\mu_{\beta'}g_{\mu\mu\mu\mu}+\hat\mu_{\beta}g_{\mu\mu\mu\beta'}+\hat\mu_{\beta\beta'}g_{\mu\mu\mu\beta}+g_{\mu\mu\beta\beta'})$$

In [22]:
def omega_bb(x, y, mu, beta_0, tau = 1):
    result = 3/4 * omega(x, y, mu, beta_0, tau)**5 * (mu_b(x, y, mu, beta_0, tau) * g_uuu(x, y, mu, beta_0, tau)\
                                                      + g_uub(x, y, mu, beta_0, tau)).reshape(x.shape[1],1)\
     @ (mu_b(x, y, mu, beta_0, tau) * g_uuu(x, y, mu, beta_0, tau)\
        + g_uub(x, y, mu, beta_0, tau)).reshape(1,x.shape[1]) + 1/2 * omega(x, y, mu, beta_0, tau)**3\
     * (mu_bb(x, y, mu, beta_0, tau) * g_uuu(x, y, mu, beta_0, tau)\
        + (mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1)\
           @ mu_b(x, y, mu, beta_0, tau).reshape(1,x.shape[1]) * g_uuuu(x, y, mu, beta_0, tau))\
        + mu_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1) @ g_uuub(x, y, mu, beta_0, tau).reshape(1,x.shape[1])\
        + mu_bb(x, y, mu, beta_0, tau)
       )
    return result

Notation for Hermite polynomial function $f_k(\hat\mu_{i0}, \hat\omega;\beta_0)$:

$$f_k(\hat\mu_{i0}, \hat\omega;\beta_0)=h_k\exp\{g(\hat\mu_{i0}+\sqrt{2\pi}\hat\omega x_k;\beta_0)+x_k^2\}$$

In [23]:
def f_k(k, x, y, mu, beta_0, tau = 1):
    [x_k, h_k] = hermgauss(k)
    return h_k * np.exp(g(x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau) * x_k, beta_0) + x_k**2)

In [24]:
# Plugged in log-sum-exp trick
def logsumexp(x):
    x.astype(np.float64)
    c = x.max()
    return (c + np.log(np.sum(np.exp(x - c)))).astype(np.float64)

def f_k(k, x, y, mu, beta_0, tau = 1):
    [x_k, h_k] = hermgauss(k)
#     h_k /= np.sqrt(np.pi)
#     [x_k, h_k] = [0, 1]
#     h_k = np.exp(-x_k**2)
    inside = g(x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau) * x_k, beta_0) + x_k**2
    result = np.exp(inside - logsumexp(inside))
    return result.astype(np.float64)

Notation for $\mathcal l_i$:

$$\mathcal l_i=\dfrac{1}{2}\log{(2\pi)}+\log(\hat\omega)+\log(\sum_{k=1}^lh_k\exp\left\{g(\hat\mu_{i0}+\sqrt{2\pi}\hat\omega x_k;\beta_0)+x_k^2\right\})$$

In [25]:
def l(k, x, y, mu, beta_0, tau = 1):
    [x_k, h_k] = hermgauss(k)
    result = 0.5 * np.log(2 * np.pi) + np.log(omega(x, y, mu, beta_0, tau)) + np.log(sum(f_k(k, x, y, mu, beta_0, tau)))
    return result.astype(np.float64)

Notation for $\sum_{k=1}^lf_{k_\beta}$:

$$\sum_{k=1}^lf_{k_\beta}(\hat\mu_{i0},\hat\omega;\beta_0)=\sum_{k=1}^lf_k(\hat\mu_{i0}, \hat\omega;\beta_0)g_\beta(\mu_{i0})$$

In [28]:
def f_k_b(k, x, y, mu, beta_0, tau = 1):
    [x_k, h_k] = hermgauss(k)
    result = 0
    for i in range(k):
        result += f_k(k, x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau) * x_k, beta_0, tau)[i] * g_b(x, y, mu + np.sqrt(2 * np.pi)\
                                                      * omega(x, y, mu, beta_0, tau) * x_k[i], beta_0, tau)
    return result

Notation for $f_{k_\mu}$:

$$f_{k_\mu}(\hat\mu_{i0},\hat\omega;\beta_0)=f_k(\hat\mu_{i0}, \hat\omega;\beta_0)g_\mu(\mu_{i0})$$

In [29]:
def f_k_u(k, x, y, mu, beta_0, tau = 1):
    [x_k, h_k] = hermgauss(k)
    return f_k(k, x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau) * x_k, beta_0, tau) * g_u(x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau) * x_k, beta_0, tau)

Notation for $f_{k_\omega}$:

$$f_{k_\omega}(\hat\mu_{i0},\hat\omega;\beta_0)=f_k(\hat\mu_{i0}, \hat\omega;\beta_0)g_\mu(\mu_{i0})\sqrt{2\pi}x_k$$

In [30]:
def f_k_w(k, x, y, mu, beta_0, tau = 1):
    [x_k, h_k] = hermgauss(k)
    return f_k(k, x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau) * x_k, beta_0, tau) * g_u(x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau) * x_k, beta_0, tau) * np.sqrt(2 * np.pi) * x_k

Notation for $\mathcal l_i'$:

$$\mathcal l_i'=\dfrac{\partial\mathcal l_i}{\partial\beta_0}=\dfrac{\hat\omega_\beta}{\hat\omega}+\dfrac{1}{\sum_{k=1}^lf_k}\sum_{k=1}^l(f_{k_\mu}\hat\mu_\beta+f_{k_\omega}\hat\omega_\beta+f_{k_\beta})$$

In [31]:
def l_1(k, x, y, mu, beta_0, tau = 1):
    [x_k, h_k] = hermgauss(k)
    result = omega_b(x, y, mu, beta_0, tau) / omega(x, y, mu, beta_0, tau)
    a = 1 / sum(f_k(k, x, y, mu, beta_0, tau))
    b = 0
    for i in range(k):
        b += f_k_u(k, x, y, mu, beta_0, tau)[i] * mu_b(x, y, mu, beta_0, tau)\
         + f_k_w(k, x, y, mu, beta_0, tau)[i] * omega_b(x, y, mu, beta_0, tau)
    b = b + f_k_b(k, x, y, mu, beta_0, tau)
#     if sum(f_k(k, x, y, mu, beta_0, tau)) > 0:
    result = result + a * b
#     else: 
#         result = result + b
    return result
#     result = 0.5 * np.log(2 * np.pi) + np.log(omega(x, y, mu, beta_0, tau))\
#     + np.log(sum(f_k_u(k, x, y, mu, beta_0, tau) * mu_b(x, y, mu, beta_0, tau)\
#                  + f_k_w(k, x, y, mu, beta_0, tau) * omega_b(x, y, mu, beta_0, tau)\
#                  + f_k_b(k, x, y, mu, beta_0, tau)) / sum(f_k(k, x, y, mu, beta_0, tau)))

In [32]:
# def l_1(k, x, y, mu, beta_0, tau = 1):
#     [x_k, h_k] = hermgauss(k)
#     result = omega_b(x, y, mu, beta_0, tau) / omega(x, y, mu, beta_0, tau)
#     b = 0
#     for i in range(k):
#         b += g_u(x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau) * x_k[i], beta_0, tau)\
#         * np.sqrt(2 * np.pi) * x_k[i]
#     result += b + g_u(x, y, mu, beta_0, tau) * mu_b(x, y, mu, beta_0, tau) + g_b(x, y, mu, beta_0, tau)
#     return result

In [29]:
beta = np.repeat(0.2, 10).reshape(10,1)
mu = [0.1, 0.1]
l_1(1, X1, y1, mu[0], beta)
Pi(X1[0],beta,mu[0])**2
g_uub(X1, y1, mu[0], beta)

NameError: name 'l_1' is not defined

Notation for $\dfrac{\partial}{\partial\beta_0}f_{k_\mu}$:

$$\sum_k\dfrac{\partial}{\partial\beta_0}f_{k_\mu}=\sum_k(\dfrac{\partial}{\partial\beta_0}(f_{k})g_\mu+f_{k_{\mu}}g_{\mu\beta'})$$

In [35]:
def f_k_ub(k, x, y, mu, beta_0, tau = 1):
    [x_k, h_k] = hermgauss(k)
    a = 0
    for i in range(k):
        a += f_k_u(k, x, y, mu, beta_0, tau)[i] * g_ub(x, y, mu, beta_0, tau)
    b = 0
    for i in range(k):
        b += f_k_u(k, x, y, mu, beta_0, tau)[i] * mu_b(x, y, mu, beta_0, tau)\
         + f_k_w(k, x, y, mu, beta_0, tau)[i] * omega_b(x, y, mu, beta_0, tau)
    b = b + f_k_b(k, x, y, mu, beta_0, tau)
    result = b * g_u(x, y, mu, beta_0, tau) + a
    return result

Notation for $\dfrac{\partial}{\partial\beta_0}f_{k_\omega}$:

$$\sum_k\dfrac{\partial}{\partial\beta_0}f_{k_\omega}=\sum_k\dfrac{\partial}{\partial\beta_0}f_{k_\mu}\cdot\sqrt{2\pi}x_k$$

In [36]:
def f_k_wb(k, x, y, mu, beta_0, tau = 1):
    [x_k, h_k] = hermgauss(k)
    a = 0
    for i in range(k):
        a += f_k_u(k, x, y, mu, beta_0, tau)[i] * g_ub(x, y, mu, beta_0, tau) * x_k[i]
    b = 0
    for i in range(k):
        b += (f_k_u(k, x, y, mu, beta_0, tau)[i] * mu_b(x, y, mu, beta_0, tau)\
         + f_k_w(k, x, y, mu, beta_0, tau)[i] * omega_b(x, y, mu, beta_0, tau)\
         + f_k(k, x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau)\
               * x_k, beta_0, tau)[i] * g_b(x, y, mu + np.sqrt(2 * np.pi)\
                                                      * omega(x, y, mu, beta_0, tau) * x_k[i], beta_0, tau)) * x_k[i]
    result = np.sqrt(2 * np.pi) * (a + b)
    return result

Notation for $\dfrac{\partial}{\partial\beta_0}f_{k_\beta}$:

$$\sum_k\dfrac{\partial}{\partial\beta_0}f_{k_\beta}=\sum_k\dfrac{\partial}{\partial\beta_0}f_{k}g_\beta +f_kg_{\beta\beta}$$

In [37]:
def f_k_bb(k, x, y, mu, beta_0, tau = 1):
    result = f_k_b(k, x, y, mu, beta_0, tau).reshape(len(beta),1) @ g_b(x, y, mu, beta_0, tau).reshape(1,len(beta_0))\
    + sum(f_k(k, x, y, mu, beta_0, tau)) * g_bb(x, y, mu, beta_0, tau)
    return result

Notation for $\sum_{k=1}^l\dfrac{\partial}{\partial\beta_0}(f_{k_\mu}\hat\mu_\beta+f_{k_\omega}\hat\omega_\beta+f_{k_\beta})$

$$\sum_{k=1}^l\dfrac{\partial}{\partial\beta_0}(f_{k_\mu}\hat\mu_\beta+f_{k_\omega}\hat\omega_\beta+f_{k_\beta})=\sum_k[\dfrac{\partial}{\partial\beta}(f_{k_\mu})\hat\mu_\beta+f_{k_\mu}\hat\mu_{\beta\beta}+\dfrac{\partial}{\partial\beta}(f_{k_\omega})\hat\omega_\beta+f_{k_\omega}\hat\omega_{\beta\beta}+\dfrac{\partial}{\partial\beta}(f_{k_\beta})]$$

In [38]:
def part_1(k, x, y, mu, beta_0, tau = 1):
    result = f_k_ub(k, x, y, mu, beta_0, tau).reshape(len(beta),1)\
    @ mu_b(x, y, mu, beta_0, tau).reshape(1,len(beta))\
    + sum(f_k_u(k, x, y, mu, beta_0, tau)) * mu_bb(x, y, mu, beta_0, tau)\
    + f_k_wb(k, x, y, mu, beta_0, tau).reshape(len(beta),1)\
    @ omega_b(x, y, mu, beta_0, tau).reshape(1,len(beta))\
    + sum(f_k_w(k, x, y, mu, beta_0, tau)) * omega_bb(x, y, mu, beta_0, tau)\
    + f_k_bb(k, x, y, mu, beta_0, tau)
    return result

Notation for $l''_i$:

$$l''_i=\dfrac{\partial^2l_i}{\partial\beta_0^2}=\dfrac{\hat\omega_{\beta\beta'}\hat\omega-\hat\omega_\beta\hat\omega_{\beta'}}{\hat\omega^2}+\dfrac{1}{\sum_{k=1}^lf_k}\sum_{k=1}^l\dfrac{\partial}{\partial\beta_0}(f_{k_\mu}\hat\mu_\beta+f_{k_\omega}\hat\omega_\beta+f_{k_\beta})-\dfrac{1}{(\sum_{k=1}^lf_k)^2}\|\sum_{k=1}^l(f_{k_\mu}\hat\mu_\beta+f_{k_\omega}\hat\omega_\beta+f_{k_\beta})\|^2_2$$

In [39]:
def l_2(k, x, y, mu, beta_0, tau = 1):
    a = 1 / sum(f_k(k, x, y, mu, beta_0, tau))
    b = 0
    for i in range(k):
        b += f_k_u(k, x, y, mu, beta_0, tau)[i] * mu_b(x, y, mu, beta_0, tau)\
         + f_k_w(k, x, y, mu, beta_0, tau)[i] * omega_b(x, y, mu, beta_0, tau)
    b = b + f_k_b(k, x, y, mu, beta_0, tau)
    part_2 = a * b
    l2 = omega(x, y, mu, beta_0, tau)**(-2) * (omega_bb(x, y, mu, beta_0, tau) * omega(x, y, mu, beta_0, tau)\
                                               - omega_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1)\
                                               @ omega_b(x, y, mu, beta_0, tau).reshape(1,x.shape[1]))\
     + sum(f_k(k, x, y, mu, beta_0, tau))**(-1) * part_1(k, x, y, mu, beta_0, tau) - part_2.reshape(x.shape[1],1)\
     @ part_2.reshape(1,x.shape[1])
    return l2

In [40]:
# def l_2(k, x, y, mu, beta_0, tau = 1):
#     [x_k, h_k] = hermgauss(k)
#     l2 = omega(x, y, mu, beta_0, tau)**(-2) * (omega_bb(x, y, mu, beta_0, tau) * omega(x, y, mu, beta_0, tau)\
#                                                - omega_b(x, y, mu, beta_0, tau).reshape(x.shape[1],1)\
#                                                @ omega_b(x, y, mu, beta_0, tau).reshape(1,x.shape[1]))
#     part1 = (g_b(x, y, mu, beta_0, tau).reshape(len(beta_0),1) * g_u(x, y, mu, beta_0, tau)\
#         @ mu_b(x, y, mu, beta_0, tau).reshape(1,len(beta_0)) + g_u(x, y, mu, beta_0, tau)\
#         * g_ub(x, y, mu, beta_0, tau).reshape(len(beta_0),1) @ omega_b(x, y, mu, beta_0, tau).reshape(1,len(beta_0))\
#         + g_u(x, y, mu, beta_0, tau) * mu_bb(x, y, mu, beta_0, tau))\
#         + g_b(x, y, mu, beta_0, tau).reshape(len(beta_0),1) @ g_b(x, y, mu, beta_0, tau).reshape(1,len(beta_0))\
#         + g_bb(x, y, mu, beta_0, tau)
#     part2 = g_u(x, y, mu, beta_0, tau) * mu_b(x, y, mu, beta_0, tau) + g_b(x, y, mu, beta_0, tau)
#     for i in range(k):
#         part1 += (g_b(x, y, mu, beta_0, tau).reshape(len(beta_0),1) * g_u(x, y, mu, beta_0, tau)\
#         @ mu_b(x, y, mu, beta_0, tau).reshape(1,len(beta_0)) + g_u(x, y, mu, beta_0, tau)\
#         * g_ub(x, y, mu, beta_0, tau).reshape(len(beta_0),1) @ omega_b(x, y, mu, beta_0, tau).reshape(1,len(beta_0))\
#         + g_u(x, y, mu, beta_0, tau) * mu_bb(x, y, mu, beta_0, tau)) * np.sqrt(2 * np.pi) * x_k[i]
        
#         part2 += g_u(x, y, mu + np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0, tau) * x_k[i], beta_0, tau)\
#         * np.sqrt(2 * np.pi) * x_k[i]
            
#     return l2 + part1 - part2.reshape(len(beta_0),1) @ part2.reshape(1,len(beta_0))

## Maximize $g(\mu_{i0})$

In [41]:
def max_mu(x, y, mu, beta_0, tau=1, max_iter=100):
    for step in range(max_iter):
#         print('Step: ', step, '\n')
        mu_new = mu - g_u(x, y, mu, beta_0, tau)/g_uu(x, y, mu, beta_0, tau)
        diff = mu_new - mu
#         print(diff)
        if np.abs(diff) < 10**(-10):
#             print(mu)
            break;
        mu = mu_new
    return mu

In [42]:
open('test.log', 'w')
beta = np.repeat(0, 10).reshape(10,1)
mu = [0.1, 0.1]
tau = 1
k = 7

print('Initial beta:', beta, "\n")
for step_mu in range(1):
    mu[0] = max_mu(X1, y1, mu[0], beta, tau)
    mu[1] = max_mu(X2, y2, mu[1], beta, tau)
    for step in range(1000):
        l1 = l_1(k, X1, y1, mu[0], beta, tau) + l_1(k, X2, y2, mu[1], beta, tau)
        l2 = l_2(k, X1, y1, mu[0], beta, tau) + l_2(k, X2, y2, mu[1], beta, tau)
        delta = l1 @ inv(l2)
        new_beta = beta - delta.reshape(10,1)
        if max(np.abs(delta)) < 10**(-10):
            break;
        beta = new_beta
        print('Step ', step+1, ':\n')
        print('Beta:\n', beta, '\n')
        print('Diff:\n', delta, '\n')
print('Beta:\n', beta, '\n')

Initial beta: [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]] 

Step  1 :

Beta:
 [[ 0.17328962]
 [-0.05512917]
 [ 0.04353724]
 [-0.22300245]
 [ 0.00094545]
 [ 0.0702574 ]
 [ 0.28235011]
 [-0.27457885]
 [ 0.12368148]
 [ 0.06985137]] 

Diff:
 [-0.17328962  0.05512917 -0.04353724  0.22300245 -0.00094545 -0.0702574
 -0.28235011  0.27457885 -0.12368148 -0.06985137] 

Step  2 :

Beta:
 [[ 0.17888404]
 [-0.05815451]
 [ 0.04708489]
 [-0.23255959]
 [ 0.00650254]
 [ 0.07616793]
 [ 0.28710386]
 [-0.2793488 ]
 [ 0.12755335]
 [ 0.06261144]] 

Diff:
 [-0.00559442  0.00302535 -0.00354765  0.00955714 -0.00555709 -0.00591054
 -0.00475375  0.00476994 -0.00387187  0.00723994] 

Step  3 :

Beta:
 [[ 0.18366945]
 [-0.05968414]
 [ 0.04915285]
 [-0.23810161]
 [ 0.00776355]
 [ 0.07935871]
 [ 0.29374591]
 [-0.28580694]
 [ 0.13075017]
 [ 0.0621852 ]] 

Diff:
 [-0.00478541  0.00152963 -0.00206796  0.00554201 -0.00126101 -0.00319077
 -0.00664205  0.00645814 -0.00319682  0.00042624] 

Step  4 :

Beta:
 [[ 0.18

Step  28 :

Beta:
 [[ 0.30770826]
 [-0.09869509]
 [ 0.08379962]
 [-0.39512726]
 [ 0.01739002]
 [ 0.13739986]
 [ 0.49182442]
 [-0.47514574]
 [ 0.21860621]
 [ 0.10070587]] 

Diff:
 [-0.00580804  0.0018181  -0.00159586  0.00721616 -0.00041034 -0.00274142
 -0.00925516  0.00877266 -0.00407384 -0.00183892] 

Step  29 :

Beta:
 [[ 0.31360536]
 [-0.10054025]
 [ 0.08541996]
 [-0.40243712]
 [ 0.01780272]
 [ 0.140186  ]
 [ 0.50121361]
 [-0.4840409 ]
 [ 0.22273645]
 [ 0.10257247]] 

Diff:
 [-0.0058971   0.00184517 -0.00162034  0.00730985 -0.00041271 -0.00278615
 -0.00938918  0.00889516 -0.00413024 -0.0018666 ] 

Step  30 :

Beta:
 [[ 0.31959361]
 [-0.1024132 ]
 [ 0.08706534]
 [-0.40984288]
 [ 0.01821775]
 [ 0.14301785]
 [ 0.5107397 ]
 [-0.4930615 ]
 [ 0.22692424]
 [ 0.10446732]] 

Diff:
 [-0.00598826  0.00187294 -0.00164539  0.00740576 -0.00041503 -0.00283185
 -0.0095261   0.00902061 -0.00418778 -0.00189485] 

Step  31 :

Beta:
 [[ 0.32567514]
 [-0.10431465]
 [ 0.08873636]
 [-0.41734681]
 [ 0.0186

Step  55 :

Beta:
 [[ 0.50568006]
 [-0.16061493]
 [ 0.13833951]
 [-0.63412724]
 [ 0.02934632]
 [ 0.23205583]
 [ 0.80339207]
 [-0.76965361]
 [ 0.35437149]
 [ 0.16307222]] 

Diff:
 [-0.00904655  0.0028376  -0.00250619  0.01074711 -0.00047588 -0.00437098
 -0.01412515  0.01336698 -0.00611151 -0.0028446 ] 

Step  56 :

Beta:
 [[ 0.51488638]
 [-0.16350368]
 [ 0.14089191]
 [-0.64505709]
 [ 0.02982523]
 [ 0.23650819]
 [ 0.81776134]
 [-0.78325557]
 [ 0.36058678]
 [ 0.16596773]] 

Diff:
 [-0.00920632  0.00288875 -0.0025524   0.01092984 -0.00047891 -0.00445237
 -0.01436927  0.01360196 -0.00621529 -0.00289551] 

Step  57 :

Beta:
 [[ 0.52425607]
 [-0.16644471]
 [ 0.14349162]
 [-0.65617447]
 [ 0.03030727]
 [ 0.24104386]
 [ 0.83238064]
 [-0.79709838]
 [ 0.36690868]
 [ 0.16891546]] 

Diff:
 [-0.00936969  0.00294103 -0.00259971  0.01111738 -0.00048203 -0.00453566
 -0.0146193   0.01384281 -0.0063219  -0.00294773] 

Step  58 :

Beta:
 [[ 0.53379283]
 [-0.16943917]
 [ 0.14613978]
 [-0.66748433]
 [ 0.0307

Step  82 :

Beta:
 [[ 0.82493965]
 [-0.26091925]
 [ 0.22770133]
 [-1.01206814]
 [ 0.0438734 ]
 [ 0.3885209 ]
 [ 1.30054088]
 [-1.24222249]
 [ 0.5702067 ]
 [ 0.26425843]] 

Diff:
 [-0.01504835  0.00471565 -0.0042331   0.01780172 -0.00062629 -0.00745323
 -0.02340134  0.02230781 -0.01024313 -0.00480226] 

Step  83 :

Beta:
 [[ 0.84030108]
 [-0.26573052]
 [ 0.23202291]
 [-1.03023781]
 [ 0.04451027]
 [ 0.39613527]
 [ 1.32442444]
 [-1.26499371]
 [ 0.58067151]
 [ 0.26916238]] 

Diff:
 [-0.01536143  0.00481127 -0.00432158  0.01816967 -0.00063687 -0.00761437
 -0.02388355  0.02277122 -0.0104648  -0.00490395] 

Step  84 :

Beta:
 [[ 0.85598432]
 [-0.27063997]
 [ 0.23643535]
 [-1.04878517]
 [ 0.04515833]
 [ 0.40391524]
 [ 1.3488031 ]
 [-1.28824066]
 [ 0.59136423]
 [ 0.27417064]] 

Diff:
 [-0.01568324  0.00490945 -0.00441244  0.01854736 -0.00064806 -0.00777997
 -0.02437866  0.02324695 -0.01069272 -0.00500825] 

Step  85 :

Beta:
 [[ 0.87199838]
 [-0.27565027]
 [ 0.24094113]
 [-1.06772026]
 [ 0.0458

Step  109 :

Beta:
 [[ 1.3853177 ]
 [-0.43592272]
 [ 0.38644599]
 [-1.67225284]
 [ 0.06774832]
 [ 0.66864397]
 [ 2.16767012]
 [-2.07143317]
 [ 0.95557612]
 [ 0.44286444]] 

Diff:
 [-0.02766596  0.00869144 -0.00796655  0.03251131 -0.00127679 -0.01388846
 -0.0426402   0.040933   -0.01913686 -0.00877533] 

Step  110 :

Beta:
 [[ 1.41369159]
 [-0.4448514 ]
 [ 0.39464064]
 [-1.70559883]
 [ 0.0690732 ]
 [ 0.68288889]
 [ 2.21139212]
 [-2.11342682]
 [ 0.97520996]
 [ 0.4518597 ]] 

Diff:
 [-0.02837389  0.00892868 -0.00819465  0.03334599 -0.00132488 -0.01424493
 -0.043722    0.04199365 -0.01963384 -0.00899526] 

Step  111 :

Beta:
 [[ 1.4428013 ]
 [-0.45402824]
 [ 0.40307453]
 [-1.73981429]
 [ 0.07044889]
 [ 0.6975037 ]
 [ 2.25623969]
 [-2.15652536]
 [ 0.99536019]
 [ 0.46108371]] 

Diff:
 [-0.02910971  0.00917684 -0.0084339   0.03421546 -0.00137569 -0.01461481
 -0.04484756  0.04309854 -0.02015023 -0.00922401] 

Step  112 :

Beta:
 [[ 1.47267702]
 [-0.46346503]
 [ 0.41175971]
 [-1.7749369 ]
 [ 0.

Step  136 :

Beta:
 [[ 2.62386936]
 [-0.84002612]
 [ 0.76805577]
 [-3.13943214]
 [ 0.13731405]
 [ 1.28384998]
 [ 4.07884598]
 [-3.92415407]
 [ 1.81233524]
 [ 0.83515342]] 

Diff:
 [-0.0827794   0.02744543 -0.02634623  0.09878366 -0.00504407 -0.04042132
 -0.1282977   0.12476281 -0.05692855 -0.02634796] 

Step  137 :

Beta:
 [[ 2.7142497 ]
 [-0.8699287 ]
 [ 0.79671435]
 [-3.24736046]
 [ 0.14280499]
 [ 1.32786601]
 [ 4.21906609]
 [-4.06039203]
 [ 1.87442585]
 [ 0.86395526]] 

Diff:
 [-0.09038034  0.02990258 -0.02865858  0.10792832 -0.00549094 -0.04401604
 -0.14022011  0.13623796 -0.06209061 -0.02880184] 

Step  138 :

Beta:
 [[ 2.81379019]
 [-0.90275812]
 [ 0.82808008]
 [-3.36630258]
 [ 0.14881816]
 [ 1.37621428]
 [ 4.37366672]
 [-4.21042599]
 [ 1.94273897]
 [ 0.89571522]] 

Diff:
 [-0.09954049  0.03282942 -0.03136573  0.11894212 -0.00601316 -0.04834827
 -0.15460063  0.15003396 -0.06831313 -0.03175996] 

Step  139 :

Beta:
 [[ 2.92450449]
 [-0.93911493]
 [ 0.86264207]
 [-3.49866855]
 [ 0.

<ipython-input-15-1c8f13c4e622>:5: RuntimeWarning: overflow encountered in power
  / (1 + np.exp(x @ beta_0 + mu))**4)\


Step  150 :

Beta:
 [[ 5.50675146]
 [-1.75314363]
 [ 1.56410058]
 [-6.58314288]
 [ 0.2935241 ]
 [ 2.68470179]
 [ 8.57222336]
 [-8.22836215]
 [ 3.78493117]
 [ 1.74942537]] 

Diff:
 [-0.086907    0.02642362 -0.01468322  0.10295595 -0.00331515 -0.04449318
 -0.13637621  0.12563066 -0.0593711  -0.02589978] 

Step  151 :

Beta:
 [[ 5.49293472]
 [-1.74775912]
 [ 1.5551784 ]
 [-6.56525142]
 [ 0.29186232]
 [ 2.67894261]
 [ 8.54927644]
 [-8.20465596]
 [ 3.77521548]
 [ 1.74440021]] 

Diff:
 [ 0.01381673 -0.00538451  0.00892218 -0.01789146  0.00166178  0.00575918
  0.02294692 -0.02370619  0.00971569  0.00502517] 

Step  152 :

Beta:
 [[ 5.49410615]
 [-1.74770568]
 [ 1.55405867]
 [-6.56596107]
 [ 0.29167756]
 [ 2.6796477 ]
 [ 8.54998692]
 [-8.20514456]
 [ 3.77595476]
 [ 1.744646  ]] 

Diff:
 [-1.17142849e-03 -5.34389095e-05  1.11973237e-03  7.09646095e-04
  1.84758210e-04 -7.05097592e-04 -7.10481606e-04  4.88603519e-04
 -7.39272914e-04 -2.45791120e-04] 

Step  153 :

Beta:
 [[ 5.49242226]
 [-1.7470

In [38]:
true_beta

array([[ 5.07909214],
       [-1.63387798],
       [ 1.44871559],
       [-6.0320438 ],
       [ 0.2875767 ],
       [ 2.56958458],
       [ 7.95979736],
       [-7.59135789],
       [ 3.50513771],
       [ 1.6770373 ]])

In [39]:
beta-true_beta

array([[ 0.4131116 ],
       [-0.11305706],
       [ 0.10402528],
       [-0.53137216],
       [ 0.0039128 ],
       [ 0.10924454],
       [ 0.58687898],
       [-0.61035703],
       [ 0.26945002],
       [ 0.06688509]])

# Significance of params

In [276]:
col = []
for i in range(10): 
    col += ['V' + str(i+1)]

X = np.concatenate([X1, X2])

V = np.diagflat(Pi(X, beta, 0) * (1 - Pi(X, beta, 0)))

SE = np.sqrt(np.diag(inv(np.transpose(X) @ V @ X))).reshape(10,1)

Z = beta/SE

P = 2 * norm.cdf(-1 * np.abs(Z))

CI_025  = beta - 1.959964 * SE
CI_975  = beta + 1.959964 * SE

df = pd.DataFrame({'Truth': np.transpose(true_beta)[0], 'Coef': np.transpose(beta)[0], 
                   'Diff': np.transpose(true_beta)[0] - np.transpose(beta)[0], 'Std.Err': np.transpose(SE)[0], 
                   'z': np.transpose(Z)[0], 'P-value': np.transpose(P)[0], 
                   '[0.025': np.transpose(CI_025)[0], '0.975]': np.transpose(CI_975)[0]},
                  index = col)
df

,Truth,Coef,Diff,Std.Err,z,P-value,[0.025,0.975]
V1,5.079092,5.492204,-0.413112,0.895722,6.131593,8.700323e-10,3.736621,7.247787
V2,-1.633878,-1.746935,0.113057,0.294856,-5.924700,3.128674e-09,-2.324843,-1.169027
V3,1.448716,1.552741,-0.104025,0.267800,5.798137,6.705550e-09,1.027863,2.077619
V4,-6.032044,-6.563416,0.531372,1.053823,-6.228198,4.718312e-10,-8.628870,-4.497961
V5,0.287577,0.291490,-0.003913,0.099078,2.942027,3.260719e-03,0.097301,0.485678
V6,2.569585,2.678829,-0.109245,0.448044,5.978937,2.245983e-09,1.800678,3.556980
V7,7.959797,8.546676,-0.586879,1.373674,6.221767,4.915874e-10,5.854326,11.239027
V8,-7.591358,-8.201715,0.610357,1.321355,-6.207048,5.398908e-10,-10.791523,-5.611906
V9,3.505138,3.774588,-0.269450,0.609624,6.191670,5.953013e-10,2.579748,4.969428
V10,1.677037,1.743922,-0.066885,0.294000,5.931718,2.997816e-09,1.167694,2.320151


In [275]:
htop

NameError: name 'htop' is not defined

## STEP 2: Maximization preparation of $\beta_0$ in LOCAL

Notation for $\dfrac{\partial\mathcal L_i}{\partial\beta_0}$

$$\dfrac{\partial\mathcal L_i}{\partial\beta_0}=\sqrt{2\pi}\hat\omega\sum_{k=1}^l\left\{f_k(\hat\mu_{i0};\beta_0)\sum_{j=1}^{n_i}(X_{ij}y_{ij}-X_{ij}\pi_{ij})\right\}$$

In [19]:
def L_1(k, x, y, mu, beta_0, tau = 1):
    L_1 = 0
    [x_k, h_k] = hermgauss(k)    
    for i in range(k):
        L_1 += np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0) *\
        f_k(k, x, y, mu, beta_0, tau = tau)[i] *\
        np.sum(x * y - x * Pi(x, beta_0, mu + np.sqrt(2 * np.pi) *\
                              omega(x, y, mu, beta_0, tau) * x_k[i]), axis=0)
    return L_1

Notation for $\dfrac{\partial^2\mathcal L_i}{\partial\beta_0^2}$:

$$\dfrac{\partial^2\mathcal L_i}{\partial\beta_0^2}=\sqrt{2\pi}\hat\omega\sum_{k=1}^l\left\{f_k(\hat\mu_{i0};\beta_0)\sum_{j=1}^{n_i}(X_{ij}y_{ij}-X_{ij}\pi_{ij})\left[\sum_{j=1}^{n_i}(X_{ij}y_{ij}-X_{ij}\pi_{ij})\right]^\top+f_k(\hat\mu_{i0};\beta_0)\sum_{j=1}^{n_i}\left(-X_{ij}\dfrac{\partial\pi_{ij}}{\partial\beta_0}\right)\right\}$$

In [21]:
def L_2(k, x, y, mu, beta_0, tau = 1):
    L2 = 0
    [x_k, h_k] = hermgauss(k) 
    for i in range(k):
        boogie = 0
        for j in range(x.shape[0]):
            boogie += -x[j].reshape(x[j].shape[0],1) @\
            Pi_1(x[j], beta_0, mu + np.sqrt(2 * np.pi) *\
                 omega(x, y, mu, beta_0, tau) * x_k[i]).reshape(1,x[j].shape[0])
        L2 += np.sqrt(2 * np.pi) * omega(x, y, mu, beta_0) *\
        (f_k(k, x, y, mu, beta_0, tau = tau)[i] *\
         np.sum(x * y - x * Pi(x, beta_0, mu + np.sqrt(2 * np.pi) *\
                               omega(x, y, mu, beta_0, tau) * x_k[i]), axis=0).reshape(x.shape[1],1) @\
         np.sum(x * y - x * Pi(x, beta_0, mu + np.sqrt(2 * np.pi) *\
                               omega(x, y, mu, beta_0, tau) * x_k[i]), axis=0).reshape(1,x.shape[1]) +\
        f_k(k, x, y, mu, beta_0, tau = tau)[i] * boogie)
    return L2

## STEP 3: Maximization of $\beta_0$ in GLOBAL

In [23]:
def LL_1(site_num, k, X, Y, mu, beta_0, tau = 1):
    LL1 = 0
    for i in range(site_num):
        LL1 += L_1(k, X[i], Y[i], mu[i], beta_0, tau) / L(k, X[i], Y[i], mu[i], beta_0, tau)
    return LL1

In [24]:
def LL_2(site_num, k, X, Y, mu, beta_0, tau = 1):
    LL2 = 0
    for i in range(site_num):
        LL2 += L_2(k, X[i], Y[i], mu[i], beta_0, tau) / L(k, X[i], Y[i], mu[i], beta_0, tau)
    LL1 = LL_1(site_num, k, X, Y, mu, beta_0, tau)
    return LL2 - LL1.reshape(LL1.shape[0],1) @ LL1.reshape(1,LL1.shape[0])

In [25]:
def update(site_num, k, X, Y, mu, beta_0, tau = 1):
    direction = LL_1(site_num, k, X, Y, mu, beta_0, tau).reshape(1,beta_0.shape[0]) @\
    inv(LL_2(site_num, k, X, Y, mu, beta_0, tau))
    return direction

# Simulation starts below

### Main function

In [225]:
beta_0 = np.repeat(0, 10).reshape(10,1)
# betea_0 = true_beta
mu = [0.1, 0.1]
X = [X1, X2]
Y = [y1, y2]
k = 1
tau = 1
print('Initial beta:', beta_0, "\n")
for step in range(10):
    print('Step ', step+1, ':\n')
    for i in range(len(mu)): 
        mu[i] = max_mu(X[i], Y[i], mu[i], beta_0)
    for substep in range(100):
        print('Mu:\n', mu, '\n')
        new_beta = beta_0 - update(len(mu), k, X, Y, mu, beta_0).reshape(beta_0.shape[0],1)
        delta = new_beta - beta_0
        beta_0 = new_beta
        print('Beta:\n', beta_0, '\n')
        print('Delta:\n', delta, '\n')

Initial beta: [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]] 

Step  1 :

Mu:
 [array([-0.1236627]), array([-0.0358604])] 



NameError: name 'update' is not defined

# Generated the data into files for R

In [981]:
np.savetxt('X1.csv', X1, delimiter=",")
np.savetxt('y1.csv', y1, delimiter=",")
np.savetxt('X2.csv', X2, delimiter=",")
np.savetxt('y2.csv', y2, delimiter=",")

# The followings are NOT true

## Step 1: Combine data to generate golden rules

In [564]:
col = []
for i in range(10): col += ['V'+str(i+1)]
X1 = pd.DataFrame(X1, columns=col)
X2 = pd.DataFrame(X2, columns=col)
# X = pd.concat([X1, X2], ignore_index = True)
y = np.concatenate((y1, y2))

Notation for $\pi_{ij}$:

$$\pi_{ij} = \dfrac{\exp{(X_{ij}^\top\beta_0})}{1 + \exp{(X_{ij}^\top\beta_0})}$$

In [566]:
def Pi(x, beta_0):
    return np.asarray((np.exp(x @ beta_0) / (1 + np.exp(x @ beta_0)))) #need to add /mu_{i0}?

Notation for $L_1$:

$$L_1 = \sum_{j=1}^{n_i}(y_{ij}X_{ij}-\pi_{ij}X_{ij})$$

In [567]:
def L_1(x, y, beta_0):
    return np.expand_dims(np.asarray((y * x - Pi(x, beta_0) * x).sum(axis = 0)), axis=1)

Notation for $L_2$:

$$L_2 = - \sum_{j=1}^{n_i}[\pi_{ij}(1-\pi_{ij})X_{ij}X_{ij}^\top]$$

In [568]:
def L_2(x, y, beta_0):
    XX = 0
    p = Pi(x, beta_0) * (1- Pi(x, beta_0))
    for i in range(x.shape[0]):
#         XX += p[i] * ((np.asarray(x.iloc[i,:]).reshape(x.shape[1],1)))**2 
        XX += p[i] * (np.asarray(x.iloc[i,:]).reshape(x.shape[1],1) \
        @ np.asarray(x.iloc[i,:]).reshape(x.shape[1],1).transpose())
    L2 = -XX
    return L2

In [588]:
beta_0 = np.repeat(0, 10).reshape(10,1)

In [1111]:
print('Initial beta:', beta_0, "\n")
for i in range(100):
    print('Step ', i+1, ':\n')
    beta_0
    beta_new = beta_0 - inv(L_2(X1, y1, beta_0) + L_2(X2, y2, beta_0)) \
    @ (L_1(X1, y1, beta_0) + L_1(X2, y2, beta_0))
    delta = beta_new - beta_0
    beta_0 = beta_new
    print('Beta:\n', beta_0, '\n')
    print('Delta:\n', delta, '\n')
    if (max(np.abs(delta)) == 0):
        break;

Initial beta: [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]] 

Step  1 :



TypeError: L_2() missing 2 required positional arguments: 'mu' and 'beta_0'

In [591]:
L_2(X1, y1, beta_0)+L_2(X2, y2, beta_0)

array([[-2017.92540591,   278.96089871,    55.13581563,    42.00608349,
           48.96279807,   210.41011784,   147.62714793,   352.95317426,
          260.13291691,    53.38371126],
       [  278.96089871, -1882.57363095,   104.82882103,   -71.34377803,
          332.49827995,   -40.34237896,   356.65541555,   265.13382507,
          523.47528537,   316.84156834],
       [   55.13581563,   104.82882103, -2018.44299488,   -17.02955605,
          -58.167005  ,    96.91290635,   -21.95912822,    81.27766099,
          252.23937134,   165.53816921],
       [   42.00608349,   -71.34377803,   -17.02955605, -1945.83029471,
           63.59659857,    38.07785122,    34.52403033,    49.45718014,
           84.64072536,    18.7808896 ],
       [   48.96279807,   332.49827995,   -58.167005  ,    63.59659857,
        -1959.36670765,    62.41459172,    57.82157479,   327.73823271,
          304.5362868 ,   118.92167227],
       [  210.41011784,   -40.34237896,    96.91290635,    38.07785122,
   

In [550]:
L_2(X1, y1, beta_0)

array([[-4140.35624389],
       [-4107.14077318],
       [-4096.38366046],
       [-4070.5526563 ],
       [-3959.86346214],
       [-4006.78389039],
       [-3930.36488249],
       [-4527.7504776 ],
       [-3984.77596394],
       [-4300.38682221]])

In [565]:
true_beta

array([[-0.0217251 ],
       [ 0.35104993],
       [-2.04337875],
       [-5.75784864],
       [ 1.19368298],
       [-5.78128915],
       [-2.31044591],
       [-2.13129101],
       [ 2.56380536],
       [ 3.40715245]])

In [587]:
inv(L_2(X1, y1, beta_0) + L_2(X2, y2, beta_0)) @ (L_1(X1, y1, beta_0) + L_1(X2, y2, beta_0))

array([[-0.10817893],
       [-0.15577158],
       [-0.04324669],
       [-0.0087236 ],
       [-0.11681485],
       [-0.05317466],
       [-0.07200324],
       [-0.17340093],
       [-0.20606506],
       [-0.11675683]])